# Section3：GRU

LSTMの課題

- 課題 \
LSTMでは、パラメータ数が多く、計算負荷が高くなる問題があった。 \
　⇒GRU
 
- GRUとは？ \
従来のLSTMでは、パラメータが多数存在していたため、計算負荷が大きかった。しかし、GRUでは、そのパラメータを大幅に削減し、精度は同等またはそれ以上が望める様になった構造。

- メリット \
計算負荷が低い。

#### ゼロから作るディープラーニング２　付録C　GRU
６章では「ゲート付きRNN」としてLSTMを詳しく説明しました。LSTMは大変良いレイヤですが、パラメータが多く、計算には時間がかかります。そこで最近では、LSTMに代わる「ゲート付きRNN」が数多く提案されています。ここではGRU（Gated Recurrent Unit）と呼ばれる有名で実績のあるゲート付きRNNで、パラメータを削減し、計算時間を短縮します。

GRUは、LSTMを”よりシンプルに”したアーキテクチャです。そのため、LSTMに比べて計算コストを抑え、パラメータも少なくすることができます。

# 実装演習

In [21]:
# coding: utf-8
import sys
sys.path.append('.')
import tensorflow.compat.v1 as tf   ## v1に切り替え
tf.disable_v2_behavior()             ## v2を無効化
import numpy as np
import re
import glob
import collections
import random
import pickle
import time
import datetime
import os

In [22]:

# logging levelを変更
# tf.logging.set_verbosity(tf.logging.ERROR)
tf.logging.set_verbosity(tf.logging.ERROR)


class Corpus:
    def __init__(self):
        self.unknown_word_symbol = "<???>" # 出現回数の少ない単語は未知語として定義しておく
        self.unknown_word_threshold = 3 # 未知語と定義する単語の出現回数の閾値
        self.corpus_file = "./corpus/**/*.txt"
        self.corpus_encoding = "utf-8"
        self.dictionary_filename = "./data_for_predict/word_dict.dic"
        self.chunk_size = 5
        self.load_dict()

        words = []
        for filename in glob.glob(self.corpus_file, recursive=True):
            with open(filename, "r", encoding=self.corpus_encoding) as f:

                # word breaking
                text = f.read()
                # 全ての文字を小文字に統一し、改行をスペースに変換
                text = text.lower().replace("\n", " ")
                # 特定の文字以外の文字を空文字に置換する
                text = re.sub(r"[^a-z '\-]", "", text)
                # 複数のスペースはスペース一文字に変換
                text = re.sub(r"[ ]+", " ", text)

                # 前処理： '-' で始まる単語は無視する
                words = [ word for word in text.split() if not word.startswith("-")]


        self.data_n = len(words) - self.chunk_size
        self.data = self.seq_to_matrix(words)

    def prepare_data(self):
        """
        訓練データとテストデータを準備する。
        data_n = ( text データの総単語数 ) - chunk_size
        input: (data_n, chunk_size, vocabulary_size)
        output: (data_n, vocabulary_size)
        """

        # 入力と出力の次元テンソルを準備
        all_input = np.zeros([self.chunk_size, self.vocabulary_size, self.data_n])
        all_output = np.zeros([self.vocabulary_size, self.data_n])

        # 準備したテンソルに、コーパスの one-hot 表現(self.data) のデータを埋めていく
        # i 番目から ( i + chunk_size - 1 ) 番目までの単語が１組の入力となる
        # このときの出力は ( i + chunk_size ) 番目の単語
        for i in range(self.data_n):
            all_output[:, i] = self.data[:, i + self.chunk_size] # (i + chunk_size) 番目の単語の one-hot ベクトル
            for j in range(self.chunk_size):
                all_input[j, :, i] = self.data[:, i + self.chunk_size - j - 1]

        # 後に使うデータ形式に合わせるために転置を取る
        all_input = all_input.transpose([2, 0, 1])
        all_output = all_output.transpose()

        # 訓練データ：テストデータを 4 : 1 に分割する
        training_num = ( self.data_n * 4 ) // 5
        return all_input[:training_num], all_output[:training_num], all_input[training_num:], all_output[training_num:]


    def build_dict(self):
        # コーパス全体を見て、単語の出現回数をカウントする
        counter = collections.Counter()
        for filename in glob.glob(self.corpus_file, recursive=True):
            with open(filename, "r", encoding=self.corpus_encoding) as f:

                # word breaking
                text = f.read()
                # 全ての文字を小文字に統一し、改行をスペースに変換
                text = text.lower().replace("\n", " ")
                # 特定の文字以外の文字を空文字に置換する
                text = re.sub(r"[^a-z '\-]", "", text)
                # 複数のスペースはスペース一文字に変換
                text = re.sub(r"[ ]+", " ", text)

                # 前処理： '-' で始まる単語は無視する
                words = [word for word in text.split() if not word.startswith("-")]

                counter.update(words)

        # 出現頻度の低い単語を一つの記号にまとめる
        word_id = 0
        dictionary = {}
        for word, count in counter.items():
            if count <= self.unknown_word_threshold:
                continue

            dictionary[word] = word_id
            word_id += 1
        dictionary[self.unknown_word_symbol] = word_id

        print("総単語数：", len(dictionary))

        # 辞書を pickle を使って保存しておく
        with open(self.dictionary_filename, "wb") as f:
            pickle.dump(dictionary, f)
            print("Dictionary is saved to", self.dictionary_filename)

        self.dictionary = dictionary

        print(self.dictionary)

    def load_dict(self):
        with open(self.dictionary_filename, "rb") as f:
            self.dictionary = pickle.load(f)
            self.vocabulary_size = len(self.dictionary)
            self.input_layer_size = len(self.dictionary)
            self.output_layer_size = len(self.dictionary)
            print("総単語数: ", self.input_layer_size)

    def get_word_id(self, word):
        # print(word)
        # print(self.dictionary)
        # print(self.unknown_word_symbol)
        # print(self.dictionary[self.unknown_word_symbol])
        # print(self.dictionary.get(word, self.dictionary[self.unknown_word_symbol]))
        return self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])

    # 入力された単語を one-hot ベクトルにする
    def to_one_hot(self, word):
        index = self.get_word_id(word)
        data = np.zeros(self.vocabulary_size)
        data[index] = 1
        return data

    def seq_to_matrix(self, seq):
        print(seq)
        data = np.array([self.to_one_hot(word) for word in seq]) # (data_n, vocabulary_size)
        return data.transpose() # (vocabulary_size, data_n)


In [23]:
class Language:
    """
    input layer: self.vocabulary_size
    hidden layer: rnn_size = 30
    output layer: self.vocabulary_size
    """

    def __init__(self):
        self.corpus = Corpus()
        self.dictionary = self.corpus.dictionary
        self.vocabulary_size = len(self.dictionary) # 単語数
        self.input_layer_size = self.vocabulary_size # 入力層の数
        self.hidden_layer_size = 30 # 隠れ層の RNN ユニットの数
        self.output_layer_size = self.vocabulary_size # 出力層の数
        self.batch_size = 128 # バッチサイズ
        self.chunk_size = 5 # 展開するシーケンスの数。c_0, c_1, ..., c_(chunk_size - 1) を入力し、c_(chunk_size) 番目の単語の確率が出力される。
        self.learning_rate = 0.005 # 学習率
        self.epochs = 1000 # 学習するエポック数
        self.forget_bias = 1.0 # LSTM における忘却ゲートのバイアス
        self.model_filename = "./data_for_predict/predict_model.ckpt"
        self.unknown_word_symbol = self.corpus.unknown_word_symbol

    def inference(self, input_data, initial_state):
        """
        :param input_data: (batch_size, chunk_size, vocabulary_size) 次元のテンソル
        :param initial_state: (batch_size, hidden_layer_size) 次元の行列
        :return:
        """
        # 重みとバイアスの初期化
        hidden_w = tf.Variable(tf.truncated_normal([self.input_layer_size, self.hidden_layer_size], stddev=0.01))
        hidden_b = tf.Variable(tf.ones([self.hidden_layer_size]))
        output_w = tf.Variable(tf.truncated_normal([self.hidden_layer_size, self.output_layer_size], stddev=0.01))
        output_b = tf.Variable(tf.ones([self.output_layer_size]))

        # BasicLSTMCell, BasicRNNCell は (batch_size, hidden_layer_size) が chunk_size 数ぶんつながったリストを入力とする。
        # 現時点での入力データは (batch_size, chunk_size, input_layer_size) という３次元のテンソルなので
        # tf.transpose や tf.reshape などを駆使してテンソルのサイズを調整する。

        input_data = tf.transpose(input_data, [1, 0, 2]) # 転置。(chunk_size, batch_size, vocabulary_size)
        input_data = tf.reshape(input_data, [-1, self.input_layer_size]) # 変形。(chunk_size * batch_size, input_layer_size)
        input_data = tf.matmul(input_data, hidden_w) + hidden_b # 重みWとバイアスBを適用。 (chunk_size, batch_size, hidden_layer_size)
        input_data = tf.split(input_data, self.chunk_size, 0) # リストに分割。chunk_size * (batch_size, hidden_layer_size)

        # RNN のセルを定義する。RNN Cell の他に LSTM のセルや GRU のセルなどが利用できる。
        cell = tf.nn.rnn_cell.BasicRNNCell(self.hidden_layer_size)
        outputs, states = tf.nn.static_rnn(cell, input_data, initial_state=initial_state)
        
        # 最後に隠れ層から出力層につながる重みとバイアスを処理する
        # 最終的に softmax 関数で処理し、確率として解釈される。
        # softmax 関数はこの関数の外で定義する。
        output = tf.matmul(outputs[-1], output_w) + output_b

        return output

    def loss(self, logits, labels):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

        return cost

    def training(self, cost):
        # 今回は最適化手法として Adam を選択する。
        # ここの AdamOptimizer の部分を変えることで、Adagrad, Adadelta などの他の最適化手法を選択することができる
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(cost)

        return optimizer

    def train(self):
        # 変数などの用意
        input_data = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size])
        actual_labels = tf.placeholder("float", [None, self.output_layer_size])
        initial_state = tf.placeholder("float", [None, self.hidden_layer_size])

        prediction = self.inference(input_data, initial_state)
        cost = self.loss(prediction, actual_labels)
        optimizer = self.training(cost)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(actual_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

        # TensorBoard で可視化するため、クロスエントロピーをサマリーに追加
        tf.summary.scalar("Cross entropy: ", cost)
        summary = tf.summary.merge_all()

        # 訓練・テストデータの用意
        # corpus = Corpus()
        trX, trY, teX, teY = self.corpus.prepare_data()
        training_num = trX.shape[0]

        # ログを保存するためのディレクトリ
        timestamp = time.time()
        dirname = datetime.datetime.fromtimestamp(timestamp).strftime("%Y%m%d%H%M%S")

        # ここから実際に学習を走らせる
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            summary_writer = tf.summary.FileWriter("./log/" + dirname, sess.graph)

            # エポックを回す
            for epoch in range(self.epochs):
                step = 0
                epoch_loss = 0
                epoch_acc = 0

                # 訓練データをバッチサイズごとに分けて学習させる (= optimizer を走らせる)
                # エポックごとの損失関数の合計値や（訓練データに対する）精度も計算しておく
                while (step + 1) * self.batch_size < training_num:
                    start_idx = step * self.batch_size
                    end_idx = (step + 1) * self.batch_size

                    batch_xs = trX[start_idx:end_idx, :, :]
                    batch_ys = trY[start_idx:end_idx, :]

                    _, c, a = sess.run([optimizer, cost, accuracy],
                                       feed_dict={input_data: batch_xs,
                                                  actual_labels: batch_ys,
                                                  initial_state: np.zeros([self.batch_size, self.hidden_layer_size])
                                                  }
                                       )
                    epoch_loss += c
                    epoch_acc += a
                    step += 1

                # コンソールに損失関数の値や精度を出力しておく
                print("Epoch", epoch, "completed ouf of", self.epochs, "-- loss:", epoch_loss, " -- accuracy:",
                      epoch_acc / step)

                # Epochが終わるごとにTensorBoard用に値を保存
                summary_str = sess.run(summary, feed_dict={input_data: trX,
                                                           actual_labels: trY,
                                                           initial_state: np.zeros(
                                                               [trX.shape[0],
                                                                self.hidden_layer_size]
                                                           )
                                                           }
                                       )
                summary_writer.add_summary(summary_str, epoch)
                summary_writer.flush()

            # 学習したモデルも保存しておく
            saver = tf.train.Saver()
            saver.save(sess, self.model_filename)

            # 最後にテストデータでの精度を計算して表示する
            a = sess.run(accuracy, feed_dict={input_data: teX, actual_labels: teY,
                                              initial_state: np.zeros([teX.shape[0], self.hidden_layer_size])})
            print("Accuracy on test:", a)


    def predict(self, seq):
        """
        文章を入力したときに次に来る単語を予測する
        :param seq: 予測したい単語の直前の文字列。chunk_size 以上の単語数が必要。
        :return:
        """

        # 最初に復元したい変数をすべて定義してしまいます
        tf.reset_default_graph()
        input_data = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size])
        initial_state = tf.compat.v1.placeholder("float", [None, self.hidden_layer_size])
        prediction = tf.nn.softmax(self.inference(input_data, initial_state))
        predicted_labels = tf.argmax(prediction, 1)

        # 入力データの作成
        # seq を one-hot 表現に変換する。
        words = [word for word in seq.split() if not word.startswith("-")]
        x = np.zeros([1, self.chunk_size, self.input_layer_size])
        for i in range(self.chunk_size):
            word = seq[len(words) - self.chunk_size + i]
            index = self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])
            x[0][i][index] = 1
        feed_dict = {
            input_data: x, # (1, chunk_size, vocabulary_size)
            initial_state: np.zeros([1, self.hidden_layer_size])
        }

        # tf.Session()を用意
        with tf.Session() as sess:
            # 保存したモデルをロードする。ロード前にすべての変数を用意しておく必要がある。
            saver = tf.train.Saver()
            saver.restore(sess, self.model_filename)

            # ロードしたモデルを使って予測結果を計算
            u, v = sess.run([prediction, predicted_labels], feed_dict=feed_dict)

            keys = list(self.dictionary.keys())


            # コンソールに文字ごとの確率を表示
            for i in range(self.vocabulary_size):
                c = self.unknown_word_symbol if i == (self.vocabulary_size - 1) else keys[i]
                print(c, ":", u[0][i])

            print("Prediction:", seq + " " + ("<???>" if v[0] == (self.vocabulary_size - 1) else keys[v[0]]))

        return u[0]


In [24]:
def build_dict():
    cp = Corpus()
    cp.build_dict()


In [25]:
ln = Language()

# 学習するときに呼び出す
#ln.train()

# 保存したモデルを使って単語の予測をする
ln.predict("some of them looks like")

総単語数:  9826
['thanks', 'the', 'rt', 'which', 't', 'party', 'are', 'you', 'going', 'to', 'rt', 'joecarney', 'essential', 'php', 'techniques', 'for', 'web', 'designer', 'and', 'developers', 'httpbitlymem', 'b', 'finds', 'a', 'drawing', 'that', 'shows', 'downtown', 'knoxville', 'was', 'a', 'much', 'busier', 'place', 'in', 'httpwigspherecom', 'rt', 'ewastesuzie', 'take', 'the', 'ng', 'e-waste', 'quiz', 'see', 'what', 'you', 'know', 'httptinyurlcomkpfy', 'woodward', 'one', 'small', 'expression', 'of', 'appreciation', 'will', 'often', 'give', 'you', 'courage', 'to', 'carry', 'on', 'for', 'a', 'great', 'whilebe', 'that', 'encourager', 'to', 'others', 'days', 'to', 'a', 'better', 'white', 'paper', 'day', 'choosing', 'a', 'winning', 'topic', 'httpowlyqmx', 'turns', 'out', 'some', 'other', 'website', 'copied', 'the', 'html', 'source', 'code', 'along', 'with', 'the', 'google', 'analytics', 'tracking', 'code', 'google', "doesn't", 'seem', 'to', 'do', 'check', 'rt', 'airamericamedia', 'watch', 'brl

C:\Users\takas\AppData\Local\Temp/ipykernel_9724/4157377894.py:45: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.nn.rnn_cell.BasicRNNCell(self.hidden_layer_size)
C:\Users\takas\anaconda3\envs\ml\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:457: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
C:\Users\takas\anaconda3\envs\ml\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:460: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(


no : 2.8676952e-08
in : 3.0622095e-06
the : 8.6762106e-05
supreme : 1.4708605e-14
court : 1.4263522e-14
of : 0.00026760978
attorney : 1.637056e-14
general : 1.4341105e-14
washington : 1.4081277e-14
dc : 1.4042574e-14
united : 1.3704491e-14
states : 1.3795652e-14
wednesday : 1.3440088e-14
october : 1.5597466e-14
matter : 1.2020349e-14
came : 1.3407366e-14
on : 2.3137222e-06
for : 1.12156556e-07
oral : 1.4371088e-14
argument : 1.4009294e-14
before : 5.053861e-13
at : 3.111843e-09
am : 1.34979935e-14
appearances : 1.6618275e-14
lawrence : 1.3353211e-14
lessig : 1.3504895e-14
esq : 1.3337633e-14
stanford : 1.25500925e-14
california : 1.2612099e-14
behalf : 1.4607427e-14
petitioners : 1.2807559e-14
theodore : 1.3951883e-14
b : 1.4119951e-14
olson : 1.5296571e-14
department : 1.5725287e-14
justice : 1.5862715e-14
we'll : 1.4367388e-14
hear : 1.39724946e-14
now : 1.3860799e-14
number : 1.2299857e-08
eric : 1.448325e-14
v : 1.5021955e-14
john : 1.463886e-14
d : 1.4904881e-14
ashcroft : 1.45663

g : 1.4087832e-14
harvey : 1.4038423e-14
richard : 1.2124935e-14
patrick : 1.5670194e-14
t : 1.4602663e-14
gillen : 1.4321697e-14
thompson : 1.4521035e-14
robert : 1.2219123e-14
muise : 1.4939433e-14
eq : 1.5288347e-14
official : 1.476223e-14
reporter : 1.4017125e-14
index : 3.0168932e-14
witnesses : 1.3541212e-14
julie : 1.4781362e-14
smith : 1.6236597e-14
christy : 1.4383182e-14
rehm : 1.500815e-14
eveland : 1.3537725e-14
following : 1.4741438e-14
discussion : 1.0441662e-15
occurred : 1.489488e-14
chambers : 1.40203875e-14
record : 1.3904091e-14
versus : 1.5362447e-14
assembled : 1.4471294e-14
planned : 1.4420484e-14
issued : 1.4476593e-14
reporters : 1.326386e-14
joseph : 1.4917993e-14
maldonado : 1.4656797e-14
heidi : 1.4387736e-14
enter : 1.4690382e-14
presumably : 4.0409958e-13
orders : 1.466127e-14
collectively : 1.3679524e-14
office : 1.3115784e-14
previously : 1.4856121e-14
entered : 1.3473685e-14
motion : 1.6152332e-14
reconsideration : 1.4542404e-14
filed : 1.2906582e-14
aug

document : 1.4841676e-14
somewhere : 1.3783684e-14
observe : 1.3652833e-14
yep : 1.5994386e-14
defines : 1.383246e-14
possible : 9.905636e-12
structures : 1.6058947e-14
r- : 1.3909316e-14
format : 1.3477618e-14
n- : 1.41155356e-14
node : 1.5007721e-14
nuh : 1.34579415e-14
convert : 1.498964e-14
wanna : 1.4355172e-14
posterior : 1.4107676e-14
probabilities : 1.6849191e-14
go-there : 1.2586889e-14
raining : 1.3484381e-14
specify : 1.6247005e-14
user : 1.3867647e-14
specification : 1.4722612e-14
inputs : 1.4820236e-14
elements : 1.2795155e-14
output : 1.41496346e-14
eh : 1.5726246e-14
struct : 1.4833412e-14
pull : 1.4499032e-14
correctly : 1.3678897e-14
t- : 1.5390543e-14
i- : 1.6929727e-14
returned : 1.4740567e-14
mm-hmm : 1.4169863e-14
k- : 1.2578777e-14
transformation : 1.4413555e-14
throw : 1.467929e-14
d- : 1.4381864e-14
u- : 1.5344261e-14
info : 1.4393336e-14
conclude : 1.3697904e-14
reverse : 1.5914294e-14
tend : 1.5669718e-14
winner : 1.4065011e-14
curiosity : 1.5643618e-14
combin

...

facility : 1.473056e-14
cake : 1.4869331e-14
reduction : 1.4836158e-14
attributed : 1.5093987e-14
ingredients : 1.2725863e-14
pressures : 1.4104663e-14
advertising : 1.4588465e-14
volume : 1.5694096e-14
operating : 1.2997046e-14
continental : 1.4696267e-14
bread : 1.5325658e-14
unit : 1.6787825e-14
south : 1.4303134e-14
composite : 1.3865029e-14
trading : 1.465786e-14
pacific : 1.4791769e-14
royal : 1.4296998e-14
toronto : 1.4253948e-14
thrift : 1.3809052e-14
obtain : 1.3650802e-14
regulatory : 1.3662001e-14
transaction : 1.5917785e-14
nj : 1.4323171e-14
financing : 1.4155763e-14
requirements : 1.41326685e-14
plastic : 1.5249205e-14
displays : 1.461392e-14
toy : 1.4108483e-14
maker : 1.5332618e-14
owed : 1.573993e-14
ranger : 1.3345165e-14
company's : 1.476375e-14
adam : 1.39806515e-14
plunged : 1.3037666e-14
patch : 1.357136e-14
bankruptcy : 1.3985292e-14
reuters : 1.4006595e-14
plc : 1.3849382e-14
reupke : 1.453037e-14
resigned : 1.3899424e-14
separation : 1.3717985e-14
appointment :

halfway : 1.4782094e-14
mound : 1.4506335e-14
stimulated : 1.5271735e-14
sub-behaviors : 1.4736603e-14
exit : 1.600397e-14
wander : 1.3328808e-14
humidity : 1.6846718e-14
degradation : 1.5518531e-14
patrollers : 1.4599265e-14
cuticular : 1.3058496e-14
hydrocarbons : 1.5531325e-14
heat : 1.4662166e-14
foragers : 1.513117e-14
patroller : 1.2730329e-14
inactive : 1.4638581e-14
discovery : 1.5440262e-14
accomplished : 1.4696882e-14
gland : 1.4387736e-14
drags : 1.4391085e-14
trails : 1.3517779e-14
tasks : 1.4594198e-14
nestmate : 1.4903972e-14
warrior : 1.2862739e-14
interactive : 1.6817566e-14
pseudocode : 1.7302271e-14
sensors : 1.4427858e-14
thermometer : 1.3386337e-14
bump : 1.5384409e-14
polarization : 1.3962212e-14
compass : 1.5601485e-14
receptors : 1.4236777e-14
circular : 1.4591137e-14
cm : 1.41066e-14
wheels : 1.4883917e-14
rear : 1.4056026e-14
wheel : 1.31247425e-14
elongated : 1.3237701e-14
skirt : 1.3077738e-14
sonar : 1.3327436e-14
detectors : 1.4664795e-14
pheromones : 1.438

webinar : 1.2583649e-14
lane : 1.6112517e-14
fibrosis : 1.2775792e-14
rss : 1.4492065e-14
obama : 1.5307839e-14
irish : 1.4418229e-14
kiss : 1.3766843e-14
eu : 1.3873362e-14
horn : 1.3945073e-14
ego : 1.3131427e-14
accident : 1.441251e-14
hits : 1.4873302e-14
kidding : 1.4931544e-14
haha : 1.4567278e-14
grin : 1.3394638e-14
band : 1.3781372e-14
ash : 1.3422334e-14
hosting : 1.4441817e-14
hip : 1.4843036e-14
hop : 1.4734101e-14
cup : 1.4418258e-14
sadly : 1.3468701e-14
omg : 1.3550926e-14
fyi : 1.5080346e-14
marketers : 1.408681e-14
occupy : 1.5501345e-14
bricks : 1.35051e-14
dreamer : 1.4231944e-14
kate : 1.4570556e-14
amazon : 1.36763405e-14
comic : 1.526323e-14
fe : 1.5607585e-14
ya : 1.392859e-14
homework : 1.3881329e-14
pun : 1.4540572e-14
jason : 1.33921084e-14
wins : 1.4365032e-14
ag : 1.423189e-14
nursing : 1.3690251e-14
elvis : 1.3659891e-14
mason : 1.429637e-14
sa : 1.5052357e-14
trout : 1.5018917e-14
networking : 1.4784267e-14
blogs : 1.4619885e-14
bike : 1.6066514e-14
jessic

cram : 1.41878205e-14
classicism : 1.6113838e-14
mies : 1.3855247e-14
der : 1.364385e-14
layout : 1.3783106e-14
composition : 1.4654309e-14
corbusier : 1.5346135e-14
roof : 1.3972123e-14
glance : 1.4197322e-14
lutyens : 1.5226303e-14
delhi : 1.4014719e-14
mouse : 1.41192515e-14
shingle : 1.4116936e-14
palladio : 1.41409745e-14
railings : 1.4177e-14
lighter : 1.3487365e-14
rustic : 1.4073436e-14
firmly : 1.4179028e-14
palladian : 1.5389486e-14
baseboards : 1.5136711e-14
lean : 1.5977403e-14
handrail : 1.4485542e-14
railing : 1.4770117e-14
functionally : 1.4780432e-14
venturi : 1.427076e-14
fireplace : 1.6051537e-14
guggenheim : 1.4396712e-14
jacobsen : 1.4489576e-14
norten : 1.3769102e-14
nod : 1.4409652e-14
slide : 1.4807632e-14
shack : 1.4235093e-14
pitched : 1.4965502e-14
sliding : 1.3398367e-14
brace : 1.4258082e-14
bare : 1.37614605e-14
slides : 1.526923e-14
uncomfortable : 1.4454851e-14
dislike : 1.5308889e-14
gehry : 1.3920092e-14
asheville : 1.4090061e-14
sharply : 1.2173784e-14

array([2.8676952e-08, 3.0622095e-06, 8.6762106e-05, ..., 2.5699688e-07,
       1.9368271e-07, 2.9195839e-05], dtype=float32)

# 確認テスト

#### P88 
問） \
LSTMとCECが抱える課題について、それぞれ簡潔に述べよ。

答） \
LSTM：パラメータ数が多く、計算負荷が高くなる \
CEC：学習能力がない

#### P92
問） \
LSTMとGRUの違いを簡潔に述べよ。

答） \
LSTM：CECがある。入力ゲートと出力ゲートと忘却ゲートの３つのゲートがある。パラメータが多いため計算量が多い。　 \
GRU：CECがない。更新ゲートとリセットゲートをもつ。パラメータが少ないため計算量が少ない。

# Section4：双方向RNN

過去の情報だけでなく、未来の情報を加味することで、精度を向上させるためのモデル

実用例：文章の推敲や、機械翻訳等

# 演習チャレンジ

In [28]:
def bindirectional_rnn_net(xs, W_f, U_f, W_b, U_b, V):
    
    xs_f = np.zeros_like(xs)
    xs_b = np.zeros_like(xs)

    for i, x in enumerate(xs):
        xs_f[i] = x
        xs_b[i] = x[::-1]

    hs_f = _rnn(xs_f, W_f, U_f)
    hs_b = _rnn(xs_b, W_b, U_b)
    hs = [np.concatenate([h_f, h_b[::-1]], axis=0)] for h_f, h_b in zip(hs_f, hs_b)
    ys = hs.dot(V.T)
    return ys


SyntaxError: invalid syntax (Temp/ipykernel_9724/916596466.py, line 14)